[코드 10.2.1]

In [1]:
# prerequisites
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 100
hidden_size = 256
image_size = 784
num_epochs = 10
BATCH_SIZE = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)

# Data Loader 
torch.manual_seed(0)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

D.to(device)
G.to(device)

# loss function
loss_function = nn.BCELoss()

# optimizer
learning_rate = 0.0002 
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate)
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate)

D_loss_epochs, G_loss_epochs = [], []
for epoch in range(num_epochs):    
    for i, (images, _) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(BATCH_SIZE, 1).to(device)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(device)

        # ======================================================== #
        #                      Train the discriminator             #
        # ======================================================== #

        outputs = D(images)
        D_loss_real = loss_function(outputs, real_labels)
                
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(BATCH_SIZE, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        D_loss_fake = loss_function(outputs, fake_labels)
                
        # Backprop and optimize
        D_loss = D_loss_real + D_loss_fake
        D_optimizer.zero_grad()       
        D_loss.backward()
        D_optimizer.step()
        
        # ========================================================= #
        #                        Train the generator                #
        # ========================================================= #

        # Compute loss with fake images
        z = torch.randn(BATCH_SIZE, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
    
        G_loss = loss_function(outputs, real_labels)
        
        # Backprop and optimize
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()      
        
    D_loss_epochs.append(D_loss.mean().item())
    G_loss_epochs.append(G_loss.mean().item())       
    
    print('Epoch [%d/%d]: D_loss: %.4f, G_loss: %.4f' % (epoch, num_epochs, D_loss.mean().item(), G_loss.mean().item()))
    
#    print('Epoch [%d/%d]: D_loss: %.4f, G_loss: %.4f' % (epoch, num_epochs, torch.mean(torch.FloatTensor(D_loss)), 
#                                                   torch.mean(torch.FloatTensor(G_loss))))
    # real image 저장
    if (epoch+1) == num_epochs:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image((images+1)/2, os.path.join('./mnist_data/', 'real_images.png'))
    
    # 생성된 이미지 저장
    if epoch==0:
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        save_image((fake_images+1)/2, os.path.join('./mnist_data/', 'fake_images-{}.png'.format(epoch+1)))
        
    if (epoch+1) % 10==0:
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        save_image((fake_images+1)/2, os.path.join('./mnist_data/', 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/10]: D_loss: 0.0329, G_loss: 5.5229
Epoch [1/10]: D_loss: 0.0193, G_loss: 5.6498
Epoch [2/10]: D_loss: 0.5421, G_loss: 4.6169
Epoch [3/10]: D_loss: 0.3044, G_loss: 5.0722
Epoch [4/10]: D_loss: 0.4379, G_loss: 3.3249
Epoch [5/10]: D_loss: 0.0674, G_loss: 4.0474
Epoch [6/10]: D_loss: 0.1099, G_loss: 5.5388
Epoch [7/10]: D_loss: 0.0583, G_loss: 6.4264
Epoch [8/10]: D_loss: 0.3589, G_loss: 4.4488
Epoch [9/10]: D_loss: 0.1740, G_loss: 5.5944


In [ ]:
import matplotlib.pyplot as plt

plt.plot(D_loss_epochs, '-')
plt.plot(G_loss_epochs, '-')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('GAN Losses');

In [11]:
# prerequisites
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 1
BATCH_SIZE = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)

# Data Loader 
torch.manual_seed(0)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

D.to(device)
G.to(device)

# loss function
loss_function = nn.BCELoss()

# optimizer
learning_rate = 0.0002 
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate)
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate)

D_loss_epochs, G_loss_epochs = [], []
for epoch in range(num_epochs):    
    for i, (images, _) in enumerate(train_loader):
        print(images.shape) 
        images = images.reshape(BATCH_SIZE, -1).to(device)
        print(images.shape) 
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(BATCH_SIZE, 1).to(device)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(device)

        # ======================================================== #
        #                      Train the discriminator             #
        # ======================================================== #

        outputs = D(images)
        print(outputs.shape) 
        D_loss_real = loss_function(outputs, real_labels)
        print(D_loss_real)         
        #break
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(BATCH_SIZE, latent_size).to(device)
        print(z.shape)        
        fake_images = G(z)
        print(fake_images.shape)
        outputs = D(fake_images)
        print(outputs.shape)        
        D_loss_fake = loss_function(outputs, fake_labels)
        print(D_loss_fake)  
        #break
        # Backprop and optimize
        D_loss = D_loss_real + D_loss_fake
        print("D_loss =",D_loss)  
        D_optimizer.zero_grad()       
        D_loss.backward()
        D_optimizer.step()
        #break
        # ========================================================= #
        #                        Train the generator                #
        # ========================================================= #

        # Compute loss with fake images
        z = torch.randn(BATCH_SIZE, latent_size).to(device)
        print(z.shape)
        fake_images = G(z)
        print(fake_images.shape)        
        outputs = D(fake_images)
        print("D_output.shape =", outputs.shape)
        G_loss = loss_function(outputs, real_labels)
        print("G_loss =", G_loss) 
        
        # Backprop and optimize
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()      
        break
    D_loss_epochs.append(D_loss.mean().item())
    G_loss_epochs.append(G_loss.mean().item())       
    
    print('Epoch [%d/%d]: D_loss: %.4f, G_loss: %.4f' % (epoch, num_epochs, D_loss.mean().item(), G_loss.mean().item()))
    
#    print('Epoch [%d/%d]: D_loss: %.4f, G_loss: %.4f' % (epoch, num_epochs, torch.mean(torch.FloatTensor(D_loss)), 
#                                                   torch.mean(torch.FloatTensor(G_loss))))
    # real image 저장
    if (epoch+1) == num_epochs:
        print(images.size(0))
        print(images.shape)
        images = images.reshape(images.size(0), 1, 28, 28)
        print(images.shape)
        save_image((images+1)/2, os.path.join('./mnist_data/', 'real_images.png'))
    
    # 생성된 이미지 저장
    if epoch==0:
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        save_image((fake_images+1)/2, os.path.join('./mnist_data/', 'fake_images-{}.png'.format(epoch+1)))
        
    if (epoch+1) % 10==0:
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        save_image((fake_images+1)/2, os.path.join('./mnist_data/', 'fake_images-{}.png'.format(epoch+1)))

torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1])
tensor(0.6527, grad_fn=<BinaryCrossEntropyBackward0>)
torch.Size([100, 64])
torch.Size([100, 784])
torch.Size([100, 1])
tensor(0.7088, grad_fn=<BinaryCrossEntropyBackward0>)
D_loss = tensor(1.3615, grad_fn=<AddBackward0>)
torch.Size([100, 64])
torch.Size([100, 784])
D_output.shape = torch.Size([100, 1])
G_loss = tensor(0.6789, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch [0/1]: D_loss: 1.3615, G_loss: 0.6789
100
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])


In [ ]:
plt.plot(real_scores_epochs, '-')
plt.plot(fake_scores_epochs, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real Score', 'Fake score'])
plt.title('Scores');

In [ ]:
# prerequisites
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 100
hidden_size = 256
image_size = 784
num_epochs = 1
BATCH_SIZE = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader 
torch.manual_seed(0)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

D.to(device)
G.to(device)

# loss function
loss_function = nn.BCELoss()

# optimizer
learning_rate = 0.0002 
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate)
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)


D_loss_epochs, G_loss_epochs, real_scores_epochs, fake_scores_epochs = [], [], [], []
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(BATCH_SIZE, 1).to(device)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        outputs = D(images)
        D_loss_real = loss_function(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(BATCH_SIZE, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        D_loss_fake = loss_function(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        D_loss = D_loss_real + D_loss_fake
        D_optimizer.zero_grad()       
        D_loss.backward()
        D_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(BATCH_SIZE, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
    
        G_loss = loss_function(outputs, real_labels)
        
        # Backprop and optimize
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, D_loss.item(), G_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    D_loss_epochs.append(D_loss.mean().item())
    G_loss_epochs.append(G_loss.mean().item())
    real_scores_epochs.append(real_score.mean().item())            
    fake_scores_epochs.append(fake_score.mean().item())  
    
    # real image 저장
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join('./mnist_data/', 'real_images.png'))
    
    # 생성된 이미지 저장
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join('./mnist_data/', 'fake_images-{}.png'.format(epoch+1)))

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 100
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [ ]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)


In [ ]:
import torch
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST

mnist = MNIST(root='data', 
              train=True, 
              download=True)

In [ ]:
print(mnist)

In [ ]:
import numpy as np
print(mnist[0])
print(type(mnist[0][0]))
image = np.array(mnist[0][0])
print(type(image))
print(image)

In [ ]:
import torch
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST

mnist = MNIST(root='data', 
              train=True, 
              download=True,
              transform=ToTensor()   
             )

In [ ]:
import numpy as np
print(mnist[0])
print(type(mnist[0][0]))
image = np.array(mnist[0][0])
print(type(image))
print(image)

In [ ]:
import torch
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST

mnist = MNIST(root='data', 
              train=True, 
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

In [ ]:
img, label = mnist[0]
print('Label: ', label)
print(img[:,10:15,10:15])
torch.min(img), torch.max(img)

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

img_norm = denorm(img)
plt.imshow(img_norm[0], cmap='gray')
print('Label:', label)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 100
data_loader = DataLoader(mnist, batch_size, shuffle=True)

In [ ]:
for img_batch, label_batch in data_loader:
    print('first batch')
    print(img_batch.shape)
    plt.imshow(img_batch[0][0], cmap='gray')
    print(label_batch)
    break

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device 

Discriminator Network

In [ ]:
import torch.nn as nn

D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [ ]:
D.to(device)

Generator Network

In [ ]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [ ]:
y = G(torch.randn(2, latent_size))
gen_imgs = denorm(y.reshape((-1, 28,28)).detach())

In [ ]:
plt.imshow(gen_imgs[0], cmap='gray');

In [ ]:
G.to(device)

Discriminator Training

In [ ]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
def train_discriminator(images):
    # Create the labels which are later used as input for the BCE loss
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)
        
    # Loss for real images
    outputs = D(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    # Loss for fake images
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs

    # Combine losses
    d_loss = d_loss_real + d_loss_fake
    # Reset gradients
    reset_grad()
    # Compute gradients
    d_loss.backward()
    # Adjust the parameters using backprop
    d_optimizer.step()
    
    return d_loss, real_score, fake_score

Generator Training

In [ ]:
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
def train_generator():
    # Generate fake images and calculate loss
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    labels = torch.ones(batch_size, 1).to(device)
    g_loss = criterion(D(fake_images), labels)

    # Backprop and optimize
    reset_grad()
    g_loss.backward()
    g_optimizer.step()
    return g_loss, fake_images

Training the Model

In [ ]:
import os

sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [ ]:
from IPython.display import Image
from torchvision.utils import save_image

# Save some real images
for images, _ in data_loader:
    images = images.reshape(images.size(0), 1, 28, 28)
    save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'), nrow=10)
    break
   
Image(os.path.join(sample_dir, 'real_images.png'))

In [ ]:
sample_vectors = torch.randn(batch_size, latent_size).to(device)

def save_fake_images(index):
    fake_images = G(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    fake_fname = 'fake_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=10)
    
# Before training
save_fake_images(0)
Image(os.path.join(sample_dir, 'fake_images-0000.png'))

In [ ]:
%%time

num_epochs = 10
total_step = len(data_loader)
d_losses, g_losses, real_scores, fake_scores = [], [], [], []

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        # Load a batch & transform to vectors
        images = images.reshape(batch_size, -1).to(device)
        
        # Train the discriminator and generator
        d_loss, real_score, fake_score = train_discriminator(images)
        g_loss, fake_images = train_generator()
        
        # Inspect the losses
        if (i+1) % 200 == 0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        
    # Sample and save images
    save_fake_images(epoch+1)

In [ ]:
Image('./samples/fake_images-0010.png')

In [ ]:
plt.plot(d_losses, '-')
plt.plot(g_losses, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real Score', 'Fake score'])
plt.title('Scores');

### https://github.com/lyeoni/pytorch-mnist-GAN/blob/master/pytorch-mnist-GAN.ipynb

In [ ]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    
    # forward method
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))
    
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [ ]:
# build network
z_dim = 100
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)

G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

In [ ]:
G

In [ ]:
D

In [ ]:
# loss
criterion = nn.BCELoss() 

# optimizer
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

In [ ]:
def D_train(x):
    #=======================Train the discriminator=======================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(BATCH_SIZE, 1)
   
    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on facke
    z = Variable(torch.randn(BATCH_SIZE, z_dim).to(device))
    x_fake, y_fake = G(z), torch.zeros(BATCH_SIZE, 1).to(device)

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [ ]:
def G_train(x):
    #=======================Train the generator=======================#
    G.zero_grad()

    z = torch.randn(BATCH_SIZE, z_dim).to(device)
    y = torch.ones(BATCH_SIZE, 1).to(device)

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

In [ ]:
n_epoch = 10
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

In [ ]:
with torch.no_grad():
    test_z = torch.randn(bs, z_dim).to(device)
    generated = G(test_z)

    save_image(generated.view(generated.size(0), 1, 28, 28), './samples/sample_' + '.png')

### https://ws-choi.github.io/blog-kor/seminar/tutorial/mnist/pytorch/gan/GAN-%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC/

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')
print(device)

In [ ]:
# standardization code
transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.5),   
                                         std=(0.5))])  

# MNIST dataset
train_data = dsets.MNIST(root='MNIST_data/', train=True, transform=transform, download=True)
test_data  = dsets.MNIST(root='MNIST_data/', train=False, transform=transform, download=True)


In [ ]:
print(train_data)
print(len(train_data))
print(train_data[0][0].shape)
print("---------------------------------")
print(test_data)
print(len(test_data))
print(test_data[0][0].shape)

In [ ]:
BATCH_SIZE = 200
train_data_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader  = torch.utils.data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt


def imshow(img):
    img = img / 2 + 0.5     # unnormalize    
    np_img = img.numpy()    
    plt.imshow(np.transpose(np_img, (1, 2, 0)), cmap='gray' )
    plt.show()

print(train_data[0][0].shape)
imshow(train_data[0][0])    

In [ ]:
torch.manual_seed(0)
def imshow_grid(img):
    img = utils.make_grid(img)
    img = img / 2 + 0.5     # unnormalize
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1,2,0)))
    plt.show()

mini_batch_img, mini_batch_label  = next(iter(train_data_loader))
print(mini_batch_img.shape)
imshow_grid(mini_batch_img[0:16,:,:])

In [ ]:
torch.manual_seed(0)
mini_batch_img, mini_batch_label  = next(iter(train_data_loader))

mini_batch=16
for i in range(mini_batch):
    plt.subplot(2,8,i+1)
    plt.imshow(mini_batch_img[i].squeeze(), cmap='gray')
plt.show() 

In [ ]:
d_noise  = 100
d_hidden = 256

def sample_z(batch_size = 1, d_noise=100):
    return torch.randn(batch_size, d_noise, device=device)

G = nn.Sequential(
    nn.Linear(d_noise, d_hidden),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(d_hidden,d_hidden),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(d_hidden, 28*28),
    nn.Tanh()
).to(device)

# 노이즈 생성하기
z = sample_z()
# 가짜 이미지 생성하기
img_fake = G(z).view(-1,28,28)
# 이미지 출력하기
print(img_fake.shape)
imshow(img_fake.detach())
# Batch SIze만큼 노이즈 생성하여 그리드로 출력하기
z = sample_z(BATCH_SIZE)
img_fake = G(z)
imshow_grid(img_fake.detach())

In [ ]:
D = nn.Sequential(
    nn.Linear(28*28, d_hidden),
    nn.LeakyReLU(),
    nn.Dropout(0.1),
    nn.Linear(d_hidden, d_hidden),
    nn.LeakyReLU(),
    nn.Dropout(0.1),
    nn.Linear(d_hidden, 1),
    nn.Sigmoid()
).to(device)

print(G(z).shape)
print(D(G(z)).shape)
print(D(G(z)[0:5]).transpose(0,1))

In [ ]:
loss_function = nn.BCELoss()

def run_epoch(generator, discriminator, _optimizer_g, _optimizer_d):

#    generator.train()
#    discriminator.train()

    for img_batch, label_batch in train_data_loader:

        img_batch, label_batch = img_batch.to(device), label_batch.to(device)

        # ================================================  #
        # maximize V(discriminator,generator) = optimize discriminator (setting k to be 1)  #
        # ================================================  #

        # init optimizer
        _optimizer_d.zero_grad()

        p_real = discriminator(img_batch.view(-1, 28*28))
        p_fake = discriminator(generator(sample_z(BATCH_SIZE, d_noise)))

        # ================================================  #
        #    Loss computation (soley based on the paper)    #
        # ================================================  #
#        loss_real = -1 * torch.log(p_real)   # -1 for gradient ascending
#        loss_fake = -1 * torch.log(1.-p_fake) # -1 for gradient ascending
#        loss_d    = (loss_real + loss_fake).mean()
        loss_d = loss_function(p_real, p_fake)
        # ================================================  #
        #     Loss computation (based on Cross Entropy)     #
        # ================================================  #
        # loss_d = criterion(p_real, torch.ones_like(p_real).to(device)) + \    #
        #          criterion(p_fake, torch.zeros_like(p_real).to(device))       #

        # Update parameters
        loss_d.backward()
        _optimizer_d.step()

        # ================================================  #
        #        minimize V(discriminator,generator)        #
        # ================================================  #

        # init optimizer
        _optimizer_g.zero_grad()

        p_fake = discriminator(generator(sample_z(BATCH_SIZE, d_noise)))

        # ================================================  #
        #    Loss computation (soley based on the paper)    #
        # ================================================  #

        # instead of: torch.log(1.-p_fake).mean() <- explained in Section 3
        loss_g = -1 * torch.log(p_fake).mean()

        # ================================================  #
        #     Loss computation (based on Cross Entropy)     #
        # ================================================  #
        # loss_g = criterion(p_fake, torch.ones_like(p_fake).to(device)) #

        loss_g.backward()

        # Update parameters
        _optimizer_g.step()



In [ ]:
def evaluate_model(generator, discriminator):

    p_real, p_fake = 0.,0.

    generator.eval()
    discriminator.eval()

    for img_batch, label_batch in test_data_loader:

        img_batch, label_batch = img_batch.to(device), label_batch.to(device)

        with torch.autograd.no_grad():
            p_real += (torch.sum(discriminator(img_batch.view(-1, 28*28))).item())/10000.
            p_fake += (torch.sum(discriminator(generator(sample_z(BATCH_SIZE, d_noise)))).item())/10000.


    return p_real, p_fake

In [ ]:
optimizer_g = optim.Adam(G.parameters(), lr = 0.0002)
optimizer_d = optim.Adam(D.parameters(), lr = 0.0002)

p_real_trace = []
p_fake_trace = []

num_epochs = 10
for epoch in range(num_epochs):

    run_epoch(G, D, optimizer_g, optimizer_d)
    p_real, p_fake = evaluate_model(G,D)

    p_real_trace.append(p_real)
    p_fake_trace.append(p_fake)

    if((epoch+1)% 5 == 0):
        print('(epoch %i/200) p_real: %f, p_g: %f' % (epoch+1, p_real, p_fake))
        imshow_grid(G(sample_z(16)).view(-1, 1, 28, 28))

In [ ]:
def init_params(model):
    for p in model.parameters():
        if(p.dim() > 1):
            nn.init.xavier_normal_(p)
        else:
            nn.init.uniform_(p, 0.1, 0.2)

init_params(G)
init_params(D)

optimizer_g = optim.Adam(G.parameters(), lr = 0.0002)
optimizer_d = optim.Adam(D.parameters(), lr = 0.0002)

p_real_trace = []
p_fake_trace = []

num_epochs = 10
for epoch in range(num_epochs):

    run_epoch(G, D, optimizer_g, optimizer_d)
    p_real, p_fake = evaluate_model(G,D)

    p_real_trace.append(p_real)
    p_fake_trace.append(p_fake)

    if((epoch+1)% 5 == 0):
        print('(epoch %i/200) p_real: %f, p_g: %f' % (epoch+1, p_real, p_fake))
        imshow_grid(G(sample_z(16)).view(-1, 1, 28, 28))

In [ ]:
plt.plot(p_fake_trace, label='D(x_generated)')
plt.plot(p_real_trace, label='D(x_real)')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show()

### https://kingnamji.tistory.com/11

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchvision.utils import save_image

import os
import numpy as np
import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device = {device}")
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [ ]:
# 하이퍼파라미터 설정
latent_size = 100
hidden_size = 256
image_size = 784 # 28 * 28
batch_size = 100
num_epochs = 5

In [ ]:
# Image Processing
transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5], # 1 for gray scale 만약, RGB channels라면 mean=(0.5, 0.5, 0.5)
                                         std=[0.5])])  # 1 for gray scale 만약, RGB channels라면 std=(0.5, 0.5, 0.5)

# MNIST 데이터셋
mnist_train = dsets.MNIST(root='data/',
                         train=True, # 트레인 셋
                         transform=transform,
                         download=True)
mnist_test  = dsets.MNIST(root='data/', 
                          train=False,
                          transform=transform,
                          download=True)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt


def imshow(img):
    img = img / 2 + 0.5     # unnormalize      
    np_img = img.numpy()    
    plt.imshow(np.transpose(np_img, (1, 2, 0)), cmap='gray' )
    plt.show()

print(mnist_train[0][0].shape)
imshow(mnist_train[0][0])    

In [ ]:
# 데이터 로더
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, # 훈련용 데이터 로딩
                                          batch_size=batch_size,
                                          shuffle=True) # 에폭마다 데이터 섞기

In [ ]:
torch.manual_seed(0)
def imshow_grid(img):
    img = utils.make_grid(img)
    img = img / 2 + 0.5     # unnormalize
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1,2,0)))
    plt.show()

mini_batch_img, mini_batch_label  = next(iter(data_loader))
print(mini_batch_img.shape)
imshow_grid(mini_batch_img[0:16,:,:])

In [ ]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()) # Binary Cross Entropy loss 를 사용할 것이기에 sigmoid 사용!

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [ ]:
print(G)
print(D)

In [ ]:
# Device setting
D = D.to(device)
G = G.to(device)

In [ ]:
# 생성자 이용해 noise 만들기
noise = torch.randn(1, 100, device=device)
fake_image = G(noise).view(-1,28,28)
print(fake_image.shape)
imshow(fake_image.detach())

In [ ]:
# Batch SIze만큼 노이즈 생성하여 그리드로 출력
noise = torch.randn(batch_size, 100, device=device)
img_fake = G(noise)
imshow_grid(img_fake.detach())

In [ ]:
print(G(noise).shape)
print(D(G(noise)).shape)
print(D(G(noise)[0:5]).transpose(0,1))

In [ ]:
# Binary cross entropy loss and optimizer
loss_function = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)


def reset_grad(): # 가중치를 0으로 초기화
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [ ]:
d_loss_epochs, g_loss_epochs, real_scores_epochs, fake_scores_epochs = [], [], [], []
total_step = len(data_loader)

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = loss_function(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = loss_function(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()       
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
    
        g_loss = loss_function(outputs, real_labels)
        
        # Backprop and optimize
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    d_loss_epochs.append(d_loss.mean().item())
    g_loss_epochs.append(g_loss.mean().item())
    real_scores_epochs.append(real_score.mean().item())            
    fake_scores_epochs.append(fake_score.mean().item())
    # real image 저장
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # 생성된 이미지 저장
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# 생성자, 판별자 각각 모델 저장
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

In [ ]:
plt.plot(d_loss_epochs, '-')
plt.plot(g_loss_epochs, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
# plot    
plt.figure(figsize = (12, 8))
plt.xlabel('epoch')
plt.ylabel('score')
x = np.arange(num_epochs)
plt.plot(x, real_scores_epochs, 'g', label='D(x)')
plt.plot(x, fake_scores_epochs, 'b', label='D(G(z))')
plt.legend()
plt.show()

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
EPOCHS = 100
BATCH_SIZE = 128

In [ ]:
trainset = datasets.MNIST("./data", train=True, download=True, 
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
)

train_loader = torch.utils.data.DataLoader(dataset = trainset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
class GNet(nn.Module):
    def __init__(self):
        super().__init__()
        self._layers = nn.Sequential(
            nn.Linear(z_size, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),    
            nn.Linear(512, 784),
            nn.Tanh()
        )
    def forward(self, x):
        return self._layers(x)

In [ ]:
class DNet(nn.Module):
    def __init__(self):
        super().__init__()
        self._layers = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.25),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.25),    
            nn.Linear(256, 128),
            nn.LeakyReLU(0.25),
            nn.Linear(128,1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self._layers(x)

In [ ]:
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)
real_labels = torch.ones(BATCH_SIZE, 1)
fake_labels = torch.zeros(BATCH_SIZE, 1)

In [ ]:
z_size = 72

In [ ]:
D = DNet()
G = GNet()

In [ ]:
fig, axes = plt.subplots(4,3)
for epoch in range(EPOCHS):
    for i, (images, _) in enumerate(train_loader):
        
        images = images.reshape(BATCH_SIZE, -1)
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        z = torch.randn(BATCH_SIZE, z_size)
        fake_images = G(z)
        outputs = D(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
    print('Epoch[{:3d}/{:3d}] d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'.format(
        epoch, EPOCHS, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))
    z = torch.randn(BATCH_SIZE, z_size)
    fake_images = G(z)
    for row in range(4):
        for col in range(3):
            fake_images_img = np.reshape(fake_images.data.cpu().numpy()[row*4+col],(28,28))
            axis = axes[row][col]
            axis.get_xaxis().set_ticks([])
            axis.get_yaxis().set_ticks([])
            axis.imshow(fake_images_img, cmap='gray')
    
    plt.savefig('./data/DNN_GAN_{:03d}.jpg'.format(epoch)) 

In [ ]:
### 손실 함수와 최적화 기법 지정하기
# Binary Cross Entropy loss
criterion = nn.BCELoss()

# 생성자의 매개 변수를 최적화하는 Adam optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
# 구분자의 매개 변수를 최적화하는 Adam optimizer
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [ ]:
# 랜덤으로 9개만 시각화
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(mnist_train), size=(1,)).item()
    img, label = mnist_train[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.axis("off") # x축, y축 안보이게 설정
    plt.imshow(img.squeeze(), cmap="gray")
plt.show() 

In [ ]:
# 데이터 로더
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, # 훈련용 데이터 로딩
                                          batch_size=batch_size,
                                          shuffle=True) # 에폭마다 데이터 섞기

In [ ]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()) # Binary Cross Entropy loss 를 사용할 것이기에 sigmoid 사용!

In [ ]:
# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [ ]:
def imshow(img):
    img = (img+1) / 2
    img = img.squeeze() # 차원 중 사이즈 1 을 제거
    np_img = img.numpy() # 이미지 픽셀을 넘파이 배열로 변환
    plt.imshow(np_img,cmap='gray')
    plt.show()


In [ ]:
def imshow_grid(img): 
    img = utils.make_grid(img.cpu().detach()) # 이미지 그리드 생성, 이미지 출력만을 위해 cpu에 담고 추적 방이
    img = (img+1)/2
    npimg = img.numpy() # 이미지 픽셀을 넘파이 배열로 변환
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

In [ ]:
# 생성자 이용해 데이터 만들기
rand = torch.randn(1, 100)
img_1 = G(rand).view(-1,28,28)

imshow(img_1.squeeze().detach())